# Chapter 12

- Tensorflow Python API
```text
┌─────────────────┐               ┌────────────────┐
│tf.keras         │High-level     │tf.distribute   │
│tf.estimator     │API            │tf.saved_model  │
└─────────────────┘               │tf.autograph    │
                                  │tf.graph_util   │Deployment &
┌─────────────────┐               │tf.lite         │optimization
│tf.nn            │               │tf.quantization │
│tf.losses        │               │tf.tpu          │
│tf.metrics       │Low-level      │tf.xla          │
│tf.optimizers    │API            └────────────────┘
│tf.train         │
│tf.initializers  │               ┌────────────────┐
└─────────────────┘               │tf.lookup       │
                                  │tf.nest         │
┌─────────────────┐               │tf.ragged       │Special data
│tf.GradientTape  │Autodiff       │tf.sets         │structures
│tf.gradients()   │               │tf.sparse       │
└─────────────────┘               │tf.strings      │
                                  └────────────────┘
┌─────────────────┐
│tf.data          │               ┌────────────────┐
│tf.feature_column│               │tf.math         │
│tf.audio         │I/O &          │tf.linalg       │Math
│tf.image         │Preprocessing  │tf.signal       │
│tf.io            │               │tf.random       │
│tf.queue         │               │tf.bitwise      │
└─────────────────┘               └────────────────┘

┌─────────────────┐               ┌────────────────┐
│tf.summary       │Tensorboard    │tf.compat       │Miscellaneous
└─────────────────┘               │tf.config       │
                                  └────────────────┘
```

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Tensor data structures and operations

1. Data structure

|  Datatype| description |
|:----|:----|
| `tf.constant()` | Immutable tensor |
| `tf.Variable()` | Mutable tensor. Use `assign()` and related methods, and `scatter_nd_update()` to update value |
| `tf.SparseTensor` | Represents tensors with mostly zeros | 
| `tf.TensorArray` | Represents list with tensors. All tensors must have the same shape and dtype. Fixed size by default but can be made mutable | 
| `tf.RaggedTensor` | Represents list of lists of tensors, where every tensor has the same shape and dtype | 
| string tensors (`tf.string`) | Represents string data (byte strings). `tf.strings` package contains ops for byte and unicode strings. `tf.string` is atomic (the length does not appear in tensor's shape, and will only be made visible after converting to `tf.int32` unicode representation tensor). See example. | 
| Sets | Defined as regular tensors |


2. Operations

|  Operation| description |
|:----|:----|
| + | addition (calls `tf.__add__`) |
| - | subtraction (calls `tf.__sub__`) |
| * | (component wise) multiplication  (calls `tf.__mul__`) |
| @ | matrix multiplication (calls `tf.__matmul__`) |
| / | division (calls `tf.__div__`) |
| `transpose()` | Transpose a matrix | 


In [18]:
# define scaler
print(tf.constant(42))

# define matrix
print("\n", tf.constant([[1,2,3], [4,5,6]]))

# shape and dtype 
t = tf.constant([[1,2,3], [4,5,6]])
print("\n", (t.shape, t.dtype))

# tensor indexing
print("\n", t[:, 1], "\n" ,t[:, 1, tf.newaxis])

tf.Tensor(42, shape=(), dtype=int32)

 tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)

 (TensorShape([2, 3]), tf.int32)

 tf.Tensor([2 5], shape=(2,), dtype=int32) 
 tf.Tensor(
[[2]
 [5]], shape=(2, 1), dtype=int32)


In [20]:
#  ┌─ "+" calls the tf.add method
(t + 10, tf.square(t), t @ tf.transpose(t))

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[11, 12, 13],
        [14, 15, 16]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[ 1,  4,  9],
        [16, 25, 36]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[14, 32],
        [32, 77]], dtype=int32)>)

`tf.transpose()` is different from numpy `.T` attribute: `tf` creates a new tensor with the transposed data, while `numpy` `.T` attribute is only a **view** of the transposed data. 

`keras.backend` provides interface to call corresponding tensorflow ops. 

In [24]:
# from numpy array to tf tensor
a = np.array([1,2,3])
print(tf.constant(a))

# from tf tensor to numpy array
print(t.numpy())

tf.Tensor([1 2 3], shape=(3,), dtype=int64)
[[1 2 3]
 [4 5 6]]


In [8]:
# type conversion
# tf does not allow implicit type conversions. Type conversion should be done
# explicitly through tf.cast() 

try:
     tf.constant([42.]) + tf.constant([42])
except Exception as e:
     print(e)

# Use tf.cast() to cast datatypes
t1 = tf.constant([42], dtype=tf.float64)
t2 = tf.constant([42], dtype=tf.int16)
tf.cast(t1, tf.int16) + t2

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


<tf.Tensor: shape=(1,), dtype=int16, numpy=array([84], dtype=int16)>

In [21]:
# tf.constant() --> immutable
# tf.variable() --> mutable, suitable for weights and optims

g1 = tf.Variable([[1,2,3], [4,5,6]])
g2 = tf.Variable([[1,2,3], [4,5,6]])
g3 = tf.Variable([[1,2,3], [4,5,6]])
print(
    g1.assign(2 * g1), # assign the operation of * 2 to g
    g2[:, 0].assign(42), # assign value 42 to selected index
    g3.scatter_nd_update(indices = [[0,0], [1,2]],
                         updates = [100, 200]) # use scatter_nd_update to update variable
)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2,  4,  6],
       [ 8, 10, 12]], dtype=int32)> <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[42,  2,  3],
       [42,  5,  6]], dtype=int32)> <tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[100,   2,   3],
       [  4,   5, 200]], dtype=int32)>


In [56]:
# string

(tf.constant("cafè"),
 tf.constant(b"tensorflow"), # b for byte string. ASCII literals only
 tf.constant([ord(c) for c in "cafè"]) # Unicode point encoding
 )

(<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa8'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'tensorflow'>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 232], dtype=int32)>)

In [60]:
u = tf.constant([ord(c) for c in "cafè"])
uu = tf.strings.unicode_encode(tf.constant(u), "UTF-8")

(tf.strings.length(uu, unit = "UTF8_CHAR"),
 tf.strings.unicode_decode(uu, "UTF-8")
)

(<tf.Tensor: shape=(), dtype=int32, numpy=4>,
 <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 232], dtype=int32)>)